# 5.3 RNN 구현
## 5.3.1 RNN 계층 구현

In [ ]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1-h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
    

## 5.3.2 Time RNN 계층 구현

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None, None
        self.stateful = stateful  #은닉상태를 유지하는지
        
    
    def set_state(self, h):
        self.h = h 
        
    def reset_state(self):
        self.h = None
        
        
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtpye='f')
        
        if not self.stateful or self.h is None:
            self.h - np.zeros((N, H), dtype='f')
        
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs
    
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, D = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, H), dtpye='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, ;] + dh)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        for i, grad in enumerate(grads):
            self.gards[i][...] = grad
        self.dh = dh
        
        return dxs

# 5.5 RNNLM 학습과 평가

## 5.5.1 RNNLM 구현

In [3]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V)/np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.rnn_layer.reset_state()

# 5.5.3 RNNLM의 학습 코드

In [4]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
from simple_rnnlm import SimpleRnnlm

batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5
lr = 0.1
max_epoch = 100

corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
print('말뭉치의 크기: %d, 어휘 수: %d' %(corpus_size, vocab_size))

max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    for iter in range(max_iters):
        batch_x = np.empty((batch_size, time_size), dtype = 'i')
        batch_t = np.empty((batch_size, time_size), dtype = 'i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1
            
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1
        
    ppl = np.exp(total_loss / loss_count)
    print('|에폭 %d | 퍼플렉서티 %.2f'%(epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

말뭉치의 크기: 1000, 어휘 수: 418
|에폭 1 | 퍼플렉서티 390.07
|에폭 2 | 퍼플렉서티 256.52
|에폭 3 | 퍼플렉서티 224.49
|에폭 4 | 퍼플렉서티 214.84
|에폭 5 | 퍼플렉서티 206.71
|에폭 6 | 퍼플렉서티 202.82
|에폭 7 | 퍼플렉서티 198.97
|에폭 8 | 퍼플렉서티 197.03
|에폭 9 | 퍼플렉서티 191.94
|에폭 10 | 퍼플렉서티 193.78
|에폭 11 | 퍼플렉서티 190.00
|에폭 12 | 퍼플렉서티 193.14
|에폭 13 | 퍼플렉서티 190.66
|에폭 14 | 퍼플렉서티 190.97
|에폭 15 | 퍼플렉서티 190.79
|에폭 16 | 퍼플렉서티 187.05
|에폭 17 | 퍼플렉서티 184.93
|에폭 18 | 퍼플렉서티 181.89
|에폭 19 | 퍼플렉서티 183.25
|에폭 20 | 퍼플렉서티 183.90
|에폭 21 | 퍼플렉서티 181.59
|에폭 22 | 퍼플렉서티 177.47
|에폭 23 | 퍼플렉서티 175.52
|에폭 24 | 퍼플렉서티 175.90
|에폭 25 | 퍼플렉서티 173.63
|에폭 26 | 퍼플렉서티 174.19
|에폭 27 | 퍼플렉서티 168.48
|에폭 28 | 퍼플렉서티 167.85
|에폭 29 | 퍼플렉서티 165.50
|에폭 30 | 퍼플렉서티 161.37
|에폭 31 | 퍼플렉서티 158.56
|에폭 32 | 퍼플렉서티 154.75
|에폭 33 | 퍼플렉서티 155.30
|에폭 34 | 퍼플렉서티 151.40
|에폭 35 | 퍼플렉서티 149.55
|에폭 36 | 퍼플렉서티 143.46
|에폭 37 | 퍼플렉서티 139.04
|에폭 38 | 퍼플렉서티 136.18
|에폭 39 | 퍼플렉서티 129.17
|에폭 40 | 퍼플렉서티 123.23
|에폭 41 | 퍼플렉서티 126.52
|에폭 42 | 퍼플렉서티 117.12
|에폭 43 | 퍼플렉서티 111.06
|에폭 44 | 퍼플렉서티 107.48
|에폭 45 | 퍼플렉서티 1

In [5]:
import sys
sys.path.append('..')
from common.trainer import RnnlmTrainer

model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

trainer.fit(xs, ts, max_epoch, batch_size, time_size)

| epoch 1 |  repeat 1 / 19 | time 0[s] | perplexity 419.03
| epoch 2 |  repeat 1 / 19 | time 0[s] | perplexity 405.11
| epoch 3 |  repeat 1 / 19 | time 0[s] | perplexity 312.99
| epoch 4 |  repeat 1 / 19 | time 0[s] | perplexity 230.76
| epoch 5 |  repeat 1 / 19 | time 0[s] | perplexity 215.21
| epoch 6 |  repeat 1 / 19 | time 0[s] | perplexity 211.77
| epoch 7 |  repeat 1 / 19 | time 0[s] | perplexity 202.58
| epoch 8 |  repeat 1 / 19 | time 0[s] | perplexity 202.55
| epoch 9 |  repeat 1 / 19 | time 0[s] | perplexity 196.13
| epoch 10 |  repeat 1 / 19 | time 0[s] | perplexity 191.78
| epoch 11 |  repeat 1 / 19 | time 0[s] | perplexity 193.39
| epoch 12 |  repeat 1 / 19 | time 0[s] | perplexity 190.44
| epoch 13 |  repeat 1 / 19 | time 0[s] | perplexity 194.54
| epoch 14 |  repeat 1 / 19 | time 0[s] | perplexity 188.45
| epoch 15 |  repeat 1 / 19 | time 0[s] | perplexity 186.87
| epoch 16 |  repeat 1 / 19 | time 0[s] | perplexity 190.74
| epoch 17 |  repeat 1 / 19 | time 0[s] | perplex